### AES-128

明文：16字节，密钥16字节，轮数10，AES的运算是定义在$GF(2^8)$上的，其中乘法和加法操作如下：

* 加法：按位异或
* 乘法：num乘以00000010左移一位，若num最高位为1，则移位的结果再与00011011异或；更高次可重复使用上述过程得到结果。

**加密流程**

第0轮：明文与第0轮密钥做加密操作（轮密钥加）

第1-9轮：明文与第1-9轮密钥做加密操作（字节代替、行移位、列混淆、轮密钥加）

第10轮：明文与第10轮密钥做加密操作（字节代替、行移位、列混淆）

**密钥的扩展**

密钥被扩展为11个16字节的轮密钥，各用于每一轮的加密

**明文与密钥的表示**

16个字节按列排列为4 * 4的字节矩阵

**各个加密操作**

* 字节代替变换——
  * S盒变换：对状态中的每个字节，高四位作为行，低四位作为列，从S盒中去对应字节作为输出
  * 逆S盒变换类似，代替表不同
  * S盒的构造

* 行移位变换——
  * 正向行移位：状态第一、二、三、四行分别循环左移0、1、2、3个字节
  * 逆向则相反，即循环右移

* 列混淆变换——
  * 正向列混淆：对每列独立的进行操作，如图，其中乘法与加法都是定义在GF(2^8)上的。

  <img src="https://s1.ax1x.com/2018/11/15/ivImyF.md.png" width=450>

  * 逆向列混淆类似，矩阵如下：

  <img src="https://s1.ax1x.com/2018/11/15/ivI3Jx.md.png" width=450>

* 轮密钥加变换——

  * 正向/逆向：128位的状态按位与128位的轮密钥异或

**密钥扩展算法**

* 用输入密钥（16个字节，4个字）直接作为扩展密钥数组的前4个字
* 对之后的每个字$w[i]$，依赖于$w[i-1]$和$w[i-4]$
  - 若 $i \ mod \ 4 \ne 0$ ， 则$w[i] = w[i-1] ⊕ w[i-4]$
  - 否则，$w[i] = G(w[i-1])⊕w[i-4] $
  其中，G的操作如下：
  - 字循环：把四个字节循环左移一个字节
  - 字代替：使用S盒代替4个字节
  - 字代替的结果与轮常量$RC[j]$异或得到输出，其中轮常量是一个字，右边三个字节为0。$RC[j]$每轮都不同，$RC[j] = 2 * RC[j-1],RC[1]=1$


In [165]:
import numpy as np

##### $GF(2^8)$下字节的加法与乘法

In [166]:
def GF_sum(num1, num2):
    return num1 ^ num2

def GF_mul(num1, num2):
    res, tmp = 0, num1
    i = 1
    while i <= 128:
        res = res ^ (0 if (num2 & i) == 0 else tmp)
        i *= 2
        if tmp & 0b10000000: tmp = (tmp * 2) % 256 ^ 0b00011011
        else: tmp = (tmp * 2) % 256
    return res

In [167]:
GF_mul(0b01010111, 0b10000011)

193

#### 字节代替

In [106]:
S_box = np.array([
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16])

S_box_1 = np.array([
    0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
    0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
    0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
    0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
    0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
    0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
    0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
    0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
    0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
    0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
    0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
    0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
    0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
    0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
    0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d])

In [141]:
def byte_sub(to_sub, cryp=1, key_extend=0): #1表示加密，即正向字节代替
    res = to_sub.copy()
    S = S_box if cryp == 1 else S_box_1
    if key_extend:
        for i in range(len(to_sub)):
            res[i] = S[int(to_sub[i] & 0b11110000) + int(to_sub[i] & 0b00001111)]
    else:
        for i in range(len(to_sub)):
            for j in range(len(to_sub[0])):
                res[i,j] = S[int(to_sub[i,j] & 0b11110000) + int(to_sub[i,j] & 0b00001111)]
    return res

In [180]:
def show_in_hex(a,s = 1):
    if s == 1:
        for i in range(4):
            for j in range(4):
                print('%.2x'%a[i,j], end=' ')
            print(' ')
    else:
        for i in range(len(a)):
            print('%.2x'%a[i], end=' ')
        print(' ')

In [185]:
#验证
a = np.array([0xea,0x04,0x65,0x85,0x83,0x45,0x5d,0x96,
              0x5c,0x33,0x98,0xb0,0xf0,0x2d,0xad,0xc5]).reshape(4,4)
show_in_hex(a)
print('**')
show_in_hex(byte_sub(a))
print('**')
show_in_hex(byte_sub(byte_sub(a,cryp=-1)))

ea 04 65 85  
83 45 5d 96  
5c 33 98 b0  
f0 2d ad c5  
**
87 f2 4d 97  
ec 6e 4c 90  
4a c3 46 e7  
8c d8 95 a6  
**
ea 04 65 85  
83 45 5d 96  
5c 33 98 b0  
f0 2d ad c5  


#### 行移位

In [211]:
def row_shift(to_shift, cryp=1): #1表示加密，即正向行移位
    res = to_shift.copy()
    for i in range(1,4):
        if cryp == 1:
            res[i] = np.append(to_shift[i][i:], to_shift[i][:i])
        else:
            res[i] = np.append(to_shift[i][4-i:], to_shift[i][:4-i])
    return res

In [187]:
#验证
b = np.arange(16).reshape(4,4)
show_in_hex(b)
print('**')
show_in_hex(row_shift(b))
print('**')
show_in_hex(row_shift(row_shift(b),cryp=-1))

00 01 02 03  
04 05 06 07  
08 09 0a 0b  
0c 0d 0e 0f  
**
00 01 02 03  
05 06 07 04  
0a 0b 08 09  
0f 0c 0d 0e  
**
00 01 02 03  
04 05 06 07  
08 09 0a 0b  
0c 0d 0e 0f  


#### 列混淆

In [131]:
matrix = np.array([[2,3,1,1],
                   [1,2,3,1],
                   [1,1,2,3],
                   [3,1,1,2]],dtype=int)
matrix_1 =np.array([[0X0E,0X0B,0X0D,0X09],
                    [0X09,0X0E,0X0B,0X0D],
                    [0X0D,0X09,0X0E,0X0B],
                    [0X0B,0X0D,0X09,0X0E]],dtype=int)

In [132]:
def col_mix(to_mix, cryp=1):
    res = np.zeros((4,4),dtype=int)
    M = matrix if cryp == 1 else matrix_1
    for i in range(4):
        for j in range(4):
            for k in range(4):
                t =  GF_sum(res[i,j], GF_mul(M[i,k], to_mix[k,j]))
                res[i,j] = GF_sum(res[i,j], GF_mul(M[i,k], to_mix[k,j]))
    return res

In [188]:
#验证
c = np.array([[0X87,0XF2,0X4D,0X97],
                [0X6E,0X4C,0X90,0XEC],
                [0X46,0XE7,0X4A,0XC3],
                [0XA6,0X8C,0XD8,0X95]],dtype=int)
show_in_hex(c)
print('**')
show_in_hex(col_mix(c))
print('**')
show_in_hex(col_mix(col_mix(c),cryp=-1))

87 f2 4d 97  
6e 4c 90 ec  
46 e7 4a c3  
a6 8c d8 95  
**
47 40 a3 4c  
37 d4 70 9f  
94 e4 3a 42  
ed a5 a6 bc  
**
87 f2 4d 97  
6e 4c 90 ec  
46 e7 4a c3  
a6 8c d8 95  


#### 轮密钥加

In [144]:
def key_add(data1, data2, key_extend=0):
    res = data1.copy()
    if key_extend:
        for i in range(len(data1)):
            res[i] = data1[i] ^ data2[i]
    else:
        for i in range(len(data1)):
            for j in range(len(data1)):
                res[i,j] = data1[i,j] ^ data2[i,j]
    return res

In [189]:
#验证
d = np.array([[0X47,0X40,0Xa3,0X4C],
                [0X37,0XD4,0X70,0X9F],
                [0X94,0XE4,0X3A,0X42],
                [0XED,0XA5,0XA6,0XBC]],dtype=int)
e = np.array([[0XAC,0X19,0X28,0X57],
                [0X77,0XFA,0XD1,0X5C],
                [0X66,0XDC,0X29,0X00],
                [0XF3,0X21,0X41,0X6A]],dtype=int)
show_in_hex(key_add(d,e))

eb 59 8b 1b  
40 2e a1 c3  
f2 38 13 42  
1e 84 e7 d6  


#### 密钥扩展

In [155]:
RC = np.array([0x01, 0x02, 0x04, 0x08,0x10, 
               0x20,0x40, 0x80,0x1b, 0x36])
def key_extend(key): #key是一个4*4的矩阵
    res = np.zeros((44,4), dtype=int)
    res[:4] = key.T
    for i in range(4,44):
        temp = res[i - 1]
        if i % 4 == 0:
            temp = np.append(res[i - 1][1:], res[i - 1][0])
            temp = byte_sub(temp,key_extend=1)
            temp[0] = temp[0] ^ RC[i // 4 - 1]
        res[i] = key_add(temp, res[i - 4], key_extend=1)
    return res 

In [190]:
#验证
key = np.array([[0X0F,0X15,0X71,0XC9],
               [0X47,0XD9,0XE8,0X59],
               [0X0C,0XB7,0XAD,0XD6],
               [0XAF,0X7F,0X67,0X98]]).T
show_in_hex(key)
print(key)
key_all = key_extend(key)
for i in range(44):
    for j in range(4):
        print('%.2x'%key_all[i,j], end=' ')
    print(' ')

0f 47 0c af  
15 d9 b7 7f  
71 e8 ad 67  
c9 59 d6 98  
[[ 15  71  12 175]
 [ 21 217 183 127]
 [113 232 173 103]
 [201  89 214 152]]
0f 15 71 c9  
47 d9 e8 59  
0c b7 ad d6  
af 7f 67 98  
dc 90 37 b0  
9b 49 df e9  
97 fe 72 3f  
38 81 15 a7  
d2 c9 6b b7  
49 80 b4 5e  
de 7e c6 61  
e6 ff d3 c6  
c0 af df 39  
89 2f 6b 67  
57 51 ad 06  
b1 ae 7e c0  
2c 5c 65 f1  
a5 73 0e 96  
f2 22 a3 90  
43 8c dd 50  
58 9d 36 eb  
fd ee 38 7d  
0f cc 9b ed  
4c 40 46 bd  
71 c7 4c c2  
8c 29 74 bf  
83 e5 ef 52  
cf a5 a9 ef  
37 14 93 48  
bb 3d e7 f7  
38 d8 08 a5  
f7 7d a1 4a  
48 26 45 20  
f3 1b a2 d7  
cb c3 aa 72  
3c be 0b 38  
fd 0d 42 cb  
0e 16 e0 1c  
c5 d5 4a 6e  
f9 6b 41 56  
b4 8e f3 52  
ba 98 13 4e  
7f 4d 59 20  
86 26 18 76  


#### AES加密解密算法

In [218]:
def AES_cryp(data, key_all, cryp=1):
    '''
    data是4*4矩阵，key_all是11个密钥，shape=(44,4)，使用时要转置
    '''
    start = 0 if cryp == 1 else 40
    key0 = key_all[start: start + 4].T
    res = key_add(data, key0)
    for i in range(1,10):
        start = 4 * i if cryp == 1 else 40 - 4 * i
        key_i = key_all[start:start+4].T
        if cryp == -1:
            key_i = col_mix(key_i, cryp) #若是解密，先对密钥进行逆向列混淆，才可以交换列混淆和轮密钥加的顺序
        res = byte_sub(res,cryp)
        res = row_shift(res, cryp)
        res = col_mix(res,cryp)
        res = key_add(res,key_i)
    start = 40 if cryp == 1 else 0
    key_10 = key_all[start:start+4].T
    res = byte_sub(res,cryp)
    res = row_shift(res, cryp)
    res = key_add(res,key_10)
    return res

In [219]:
data = np.array([[0x01,0x23,0x45,0x67],
                [0x89,0xab,0xcd,0xef],
                [0xfe,0xdc,0xba,0x98],
                [0x76,0x54,0x32,0x10]]).T

key = np.array([[0X0F,0X15,0X71,0XC9],
               [0X47,0XD9,0XE8,0X59],
               [0X0C,0XB7,0XAD,0XD6],
               [0XAF,0X7F,0X67,0X98]]).T
key_all = key_extend(key)

In [222]:
print('明文是：')
show_in_hex(data)
C = AES_cryp(data, key_all)
P = AES_cryp(C, key_all, -1)
print('加密后：')
show_in_hex(C)
print('解密后：')
show_in_hex(P)

明文是：
01 89 fe 76  
23 ab dc 54  
45 cd ba 32  
67 ef 98 10  
加密后：
ff 08 69 64  
0b 53 34 14  
84 bf ab 8f  
4a 7c 43 b9  
解密后：
01 89 fe 76  
23 ab dc 54  
45 cd ba 32  
67 ef 98 10  


In [250]:
def get_data_16_byte(s): #把16字节的字符串转为4*4的矩阵
    res = np.zeros((4,4), dtype=int)
    for i in range(len(s)):
        res[i % 4, i // 4] = ord(s[i])
    return res

def cut_data_16(sentence):
    res = []
    i = 0
    while (i + 1) * 16 <= len(sentence):
        data = get_data_16_byte(sentence[i * 16: (i + 1) * 16])
        res.append(data)
        i += 1
    if i * 16 < len(sentence):
        data = get_data_16_byte(sentence[i * 16:])
        res.append(data)
    return res


In [251]:
def AES_encryp_decryp_sentence(sentence, key_all, cryp=1):
    data_all = cut_data_16(sentence)
    res = []
    for each in data_all:
        res.append(AES_cryp(each,key_all,cryp))
    res_sentence = ''
    for each in res:
        for i in range(4):#把4*4的矩阵转为16字节的字符串
            for j in range(4):
                res_sentence += chr(each[j,i])
                if each[j,i] == '\0': return res_sentence[:-1]
    return res_sentence

In [253]:
#确定密钥
key_str = 'This is a Secret'
key = get_data_16_byte(key_str)
key_all = key_extend(key)
#明文
sentence = 'let us make an examination, this is a sentence with length 62!'
C_sentence = AES_encryp_decryp_sentence(sentence, key_all, 1)
P_sentence = AES_encryp_decryp_sentence(C_sentence, key_all, -1)
print('明文是：', sentence)
print('加密后：', C_sentence)
print('解密后：', P_sentence)

明文是： let us make an examination, this is a sentence with length 62!
加密后： ³'ÛnS¢Ñ51À&«ñÖ®û¼:&§\É·yøÅMÉsñ´~ {£îá	´×³óAÖú¼¿
解密后： let us make an examination, this is a sentence with length 62!  
